In [1]:
import random
import numpy as np

In [2]:
'''
This is a variant of random walk. The agent can walk to any adjacent state, or the agent can teleport to a random state in hopes of getting closer to state 0.
'''

states = [
    {'terminal': True, 'reward': 25}, # 0
    {'terminal': False, 'reward': 0}, # 1
    {'terminal': False, 'reward': 0}, # 2
    {'terminal': False, 'reward': -100}, # 3
    {'terminal': False, 'reward': 0}, # 4
    {'terminal': False, 'reward': -100}, # 5
    {'terminal': False, 'reward': 0}, # 6
    {'terminal': False, 'reward': 0}, # 7
    {'terminal': True, 'reward': 25} # 8
]

DISCOUNT = .9
LEARNING_RATE = .1
CONVERGENCE_THRESHOLD = .01

In [3]:
def greedy_policy(q, state):

    max_q = max(q[state].values())
    max_actions = [action for action, value in q[state].items() if value == max_q]
    return random.choice(max_actions)


def epsilon_greedy_policy(q, state, epsilon=0.3):

    if random.random() < epsilon:
        # return random.choice(['left', 'right', 'teleport'])
        return random.choice(['left', 'right'])
    else:
        max_q = max(q[state].values())
        max_actions = [action for action, value in q[state].items() if value == max_q]
        return random.choice(max_actions)


def n_step_SARSA(states, n=2, lr=LEARNING_RATE, d=DISCOUNT, ct=CONVERGENCE_THRESHOLD, min_episode_count=10000):

    q = {}
    starting_states = []
    for i, s in enumerate(states):
        if not s['terminal']:
            q[i] = {'left': 0, 'right': 0, 'teleport': 0}
            # q[i] = {'left': 0, 'right': 0}
            starting_states.append(i)

    episode_count = 0
    while True:
        episode_count += 1
        episode = {
            's': {},
            'a': {},
            'r': {},
        }
        t = 0
        T = float('inf')

        # Select and store s_0, a_0
        state = random.choice(starting_states)
        action = epsilon_greedy_policy(q, state)
        episode['a'][t] = action
        episode['s'][t] = state
        
        while True:
            if t < T:
                # Take a_t, observe and store s_t+1 and r_t+1
                match action:
                    case 'left':
                        state -= 1
                    case 'right':
                        state += 1
                    case 'teleport':
                        state = random.randint(0, len(states)-1)
                episode['s'][t+1] = state
                reward = states[state]['reward']
                episode['r'][t+1] = reward

                if states[state]['terminal']: 
                    T = t + 1
                else:
                    # Select and store a_t+1
                    action = epsilon_greedy_policy(q, state)
                    episode['a'][t+1] = action

            tau = t - n + 1
            if tau >= 0:
                G = 0
                for i in range(tau + 1, min(tau + n, T)):
                    G += (d ** (i - tau - 1)) * episode['r'][i]
                if tau + n < T:
                    G += (d ** n) * q[episode['s'][tau + n]][episode['a'][tau + n]]

                q[episode['s'][tau]][episode['a'][tau]] += lr * (G - q[episode['s'][tau]][episode['a'][tau]])

            t += 1
            
            if tau == T - 1:
                break
        if episode_count >= min_episode_count:
            break
   
    return q

q = n_step_SARSA(states)

In [4]:
line1 = ''
line2 = ''
line3 = ''
for a in range(0, len(states)):
    line1 += str(a) + '\t'
    line2 += str(states[a]['reward']) + '\t'
    if states[a]['terminal']:
        line3 += '*'
    else:
        max_a = max(q[a], key=q[a].get)
        line3 += max_a
    line3 += '\t'  
print(line1)
print(line2)
print(line3.replace('teleport', 'T').replace('left', '<-').replace('right', '->'))

0	1	2	3	4	5	6	7	8	
25	0	0	-100	0	-100	0	0	25	
*	<-	<-	<-	T	->	->	->	*	


In [5]:
def softmax_actions(action_dict, temperature=.1):

    q_values = np.array(list(action_dict.values()))
    scaled_q = q_values / temperature
    exp_q = np.exp(scaled_q - np.max(scaled_q))
    prob = exp_q / np.sum(exp_q)
    return {'left': prob[0], 'right': prob[1], 'teleport': prob[2]}


def n_step_Tree_Backup(states, n=2, lr=LEARNING_RATE, d=DISCOUNT, ct=CONVERGENCE_THRESHOLD, min_episode_count=10000):

    q = {}
    starting_states = []
    for i, s in enumerate(states):
        if not s['terminal']:
            q[i] = {'left': 0, 'right': 0, 'teleport': 0}
            # q[i] = {'left': 0, 'right': 0}
            starting_states.append(i)

    episode_count = 0
    while True:
        episode_count += 1
        episode = {
            's': {},
            'a': {},
            'r': {},
        }
        t = 0
        T = float('inf')

        # Select and store s_0, a_0
        state = random.choice(starting_states)
        action = epsilon_greedy_policy(q, state)
        episode['a'][t] = action
        episode['s'][t] = state
        
        while True:
            if t < T:
                # Take a_t, observe and store s_t+1 and r_t+1
                match action:
                    case 'left':
                        state -= 1
                    case 'right':
                        state += 1
                    case 'teleport':
                        state = random.randint(0, len(states)-1)
                episode['s'][t+1] = state
                reward = states[state]['reward']
                episode['r'][t+1] = reward

                if states[state]['terminal']: 
                    T = t + 1
                else:
                    # Select and store a_t+1
                    action = epsilon_greedy_policy(q, state)
                    episode['a'][t+1] = action

            tau = t - n + 1
            if tau >= 0:
                if t + 1 >= T:
                    G = episode['r'][T]
                else:
                    softmax_probs = softmax_actions(q[episode['s'][t+1]])
                    softmax_G = 0
                    for a in softmax_probs.keys():
                        softmax_G += softmax_probs[a] * q[episode['s'][t+1]][a]
                    softmax_G *= d
                    G = episode['r'][t+1] + (d * softmax_G)

                for k in range(min(t, T - 1), (tau + 1) - 1, -1):
                    softmax_G_tree = 0
                    softmax_probs_tree = softmax_actions(q[episode['s'][k]])
                    for a in softmax_probs_tree.keys():
                        if a != episode['a'][k]:
                            softmax_G_tree += softmax_probs_tree[a] * q[episode['s'][k]][a]
                    G = episode['r'][k] + (d * softmax_G_tree) + (d * softmax_probs_tree[episode['a'][k]] * G)

                q[episode['s'][tau]][episode['a'][tau]] += lr * (G - q[episode['s'][tau]][episode['a'][tau]])

            t += 1
            
            if tau == T - 1:
                break
        if episode_count >= min_episode_count:
            break
   
    return q

q = n_step_Tree_Backup(states)

In [6]:
line1 = ''
line2 = ''
line3 = ''
for a in range(0, len(states)):
    line1 += str(a) + '\t'
    line2 += str(states[a]['reward']) + '\t'
    if states[a]['terminal']:
        line3 += '*'
    else:
        max_a = max(q[a], key=q[a].get)
        line3 += max_a
    line3 += '\t'  
print(line1)
print(line2)
print(line3.replace('teleport', 'T').replace('left', '<-').replace('right', '->'))

0	1	2	3	4	5	6	7	8	
25	0	0	-100	0	-100	0	0	25	
*	<-	<-	<-	T	->	->	->	*	


Notes to help digest Unifying Algorithm: n-step Q(o)

o_k: Degree of Sampling

p_k: Importance Sampling Weight (pi(A_k | S_k) / b(A_k | S_k))

The formula below may be hard to comprehend why we have the additive term:

G = R_k + y * [o_k * p_k  + (1 - o_k) * π(A_k | S_k)] * (G - Q(S_k, A_k)) + y * V

When o_k == 0:
G = R_k + y * [pi(A_k | S_k)] * (G - Q(S_k, A_k)) + y * V

When o_k == 1:
G = R_k + y * [p_k] * (G - Q(S_k, A_k)) + y * V